## 1. Import Dependencies

In [1]:
import keras
import os
import nltk
from nltk.stem import WordNetLemmatizer
import pickle
import json
import numpy as np
from keras.models import Sequential
from keras.callbacks import EarlyStopping
from keras.layers import Dense,Dropout,Activation
import random
import datetime
from googlesearch import *
import webbrowser
import requests
# from pycricbuzz import Cricbuzz
# import billboard
import time

2024-01-15 16:24:08.063369: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-01-15 16:24:08.118071: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2024-01-15 16:24:08.430134: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-01-15 16:24:08.430603: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-01-15 16:24:08.507514: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to

## 2. Constants and Paths

In [13]:
DATA_PATH = os.path.abspath(os.path.join(os.getcwd(), '..', 'data'))
MODEL_PATH = os.path.abspath(os.path.join(os.getcwd(), '..', 'models'))
INTENTS_PATH = os.path.join(DATA_PATH, 'intents.json')
WORDS_PATH = os.path.join(DATA_PATH, 'words.pkl')
CLASSES_PATH = os.path.join(DATA_PATH, 'classes.pkl')

## 3. Tokenize and Lemmatize the Intents

In [3]:
# Initializing the WordNet lemmatizer
lemmatizer=WordNetLemmatizer()


In [4]:
# Lists to store words, classes, and documents
words=[]
classes=[]
documents=[]

# Ignored characters in tokenization
ignore=['?','!',',',"'s"]

In [5]:
# Reading data from intents.json file
data_file=open(INTENTS_PATH).read()
intents=json.loads(data_file)

In [6]:
# Loop through each intent in the intents dictionary
for intent in intents['intents']:
    # Loop through each pattern in the current intent
    for pattern in intent['patterns']:
        # Tokenize the pattern into words
        w = nltk.word_tokenize(pattern)

        # Extend the words list with the tokenized words
        words.extend(w)

        # Append a tuple containing (tokenized words, intent tag) to the documents list
        documents.append((w, intent['tag']))

        # Add intent tag to classes list, if still missing
        if intent['tag'] not in classes:
            classes.append(intent['tag'])

In [7]:
# Lemmatize and lowercase each word in the words list, excluding ignored characters
words = [lemmatizer.lemmatize(word.lower()) for word in words if word not in ignore]

# Remove duplicate words, sort, and convert to a list
words = sorted(list(set(words)))

# Sort and convert classes to a list
classes = sorted(list(set(classes)))

# Save the preprocessed words and classes to pickle files for later use
pickle.dump(words,open(WORDS_PATH,'wb'))
pickle.dump(classes,open(CLASSES_PATH,'wb'))

## 4. Preparing Training Data

In [8]:
# List to store training data and initialize an empty output
training = []
output_empty = [0]*len(classes)

In [9]:
# Iterate through each document in the documents list
for doc in documents:
    # Initialize an empty bag of words for the current document
    bag = []

    # Get the tokenized pattern from the document
    pattern = doc[0]

    # Lemmatize and lowercase each word in the pattern
    pattern = [lemmatizer.lemmatize(word.lower()) for word in pattern ]

    # Iterate through each word in the list of all words
    for word in words:
        # Check if the word is in the pattern, append 1 if true, 0 if false
        if word in pattern:
            bag.append(1)
        else:
            bag.append(0)

    output_row = list(output_empty)
    # Set the value at the index corresponding to the class of the current document to 1
    output_row[classes.index(doc[1])] = 1

    # Append a pair of the bag of words and the output row to the training list
    training.append((bag, output_row))

In [10]:
# Shuffle the training data randomly
random.shuffle(training)

# Convert the training list to a NumPy array
# training = np.array(training)

# Separate the features (X_train) and labels (y_train) from the training array
# X_train = list(training[:,0])
# y_train = list(training[:,1])
X_train = np.array([item[0] for item in training])  # Convert bag to NumPy array
y_train = np.array([item[1] for item in training])  # Convert output_row to NumPy array

## 5. Modelling

In [11]:
model=Sequential()
model.add(Dense(128, activation='relu', input_shape=(len(X_train[0]),)))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(y_train[0]), activation='softmax'))

adam = keras.optimizers.Adam(0.001)
model.compile(optimizer=adam,
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [12]:
early_stopping = EarlyStopping(patience=10, restore_best_weights=True)
#model.fit(np.array(X_train),np.array(y_train),epochs=200,batch_size=10,verbose=1)
history = model.fit(X_train, y_train,
                    epochs=200,
                    batch_size=10,
                    callbacks=[early_stopping],
                    verbose=1)


Epoch 1/200
12/12 [==============================] - 1s 5ms/step - loss: 3.3178 - accuracy: 0.0357 
Epoch 2/200
12/12 [==============================] - 0s 4ms/step - loss: 3.2870 - accuracy: 0.0714
Epoch 3/200
12/12 [==============================] - 0s 4ms/step - loss: 3.2763 - accuracy: 0.0536
Epoch 4/200
12/12 [==============================] - 0s 4ms/step - loss: 3.2633 - accuracy: 0.0357
Epoch 5/200
12/12 [==============================] - 0s 4ms/step - loss: 3.2218 - accuracy: 0.1339
Epoch 6/200
12/12 [==============================] - 0s 4ms/step - loss: 3.2540 - accuracy: 0.0714
Epoch 7/200
12/12 [==============================] - 0s 4ms/step - loss: 3.1847 - accuracy: 0.1250
Epoch 8/200
12/12 [==============================] - 0s 4ms/step - loss: 3.1575 - accuracy: 0.0982
Epoch 9/200
12/12 [==============================] - 0s 4ms/step - loss: 3.0994 - accuracy: 0.1161
Epoch 10/200
12/12 [==============================] - 0s 3ms/step - loss: 3.0631 - accuracy: 0.1696
Epoch 11

In [16]:
for version in range(1, 5):
    file = f'kaybot_model_{str(version)}.h5'
    file_path = os.path.join(MODEL_PATH, file)
    if not os.path.exists(file_path):
        model.save(file)
        


/home/jarisfenner/code/Kaaykun/KayBot/models/kaybot_model_1.h5
/home/jarisfenner/code/Kaaykun/KayBot/models/kaybot_model_2.h5
/home/jarisfenner/code/Kaaykun/KayBot/models/kaybot_model_3.h5
/home/jarisfenner/code/Kaaykun/KayBot/models/kaybot_model_4.h5


In [ ]:
model.save('mymodel.h5', history)

In [ ]:

from keras.models import load_model
model = load_model('mymodel.h5')
intents = json.loads(open('intents.json').read())
words = pickle.load(open('words.pkl','rb'))
classes = pickle.load(open('classes.pkl','rb'))